In [ ]:
%load_ext google.colab.data_table

In [1]:
!pip install sshtunnel
!pip install mysqlclient
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.2.4-cp310-cp310-linux_x86_64.whl size=124733 sha256=b58bf5a43931cea88034422a05635c7ed0b04c3039f64fe67df78e8c745bdad2
  Stored in directory: /root/.cache/pip/wheels/ac/96/ac/2a4d8cb58a4d95de1dffc3f8b0ea42e0e5b63ab97640edbda3
Successfully built mysqlclient
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.9 MB/s eta 0:00:00


In [2]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import plotly.express as px

In [4]:
server = SSHTunnelForwarder(
    (SSH_HOST, 22),
    ssh_username=SSH_USERNAME,
    ssh_password=SSH_PASSWORD,
    remote_bind_address=(DB_HOST, DB_PORT),
    local_bind_address=('127.0.0.1', LOCAL_PORT)
)

In [5]:
def load_table_from_mysql(table_name):
  server.start()
  try:
    connection = pymysql.connect(
        host='127.0.0.1',
        port=LOCAL_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME
    )
    query = f"SELECT * FROM {table_name}"
    df_pandas = pd.read_sql(query, connection)
  finally:
    connection.close()
    server.stop()
  return df_pandas

def comparer_column(df_1, df_2):
    unique_slug_game = df_1.unique()
    unique_game_slug = df_2.unique()

    set_slug_game = set(unique_slug_game)
    set_game_slug = set(unique_game_slug)

    if set_slug_game.issubset(set_game_slug) and set_game_slug.issubset(set_slug_game):
        return "Toutes les valeurs correspondent"
    else:
        result = "Certaines valeurs ne correspondent pas\n"
        only_in_results = set_slug_game.difference(set_game_slug)
        if only_in_results:
            result += f"Valeurs uniquement dans df_1: {only_in_results}\n"
        only_in_hosts = set_game_slug.difference(set_slug_game)
        if only_in_hosts:
            result += f"Valeurs uniquement dans df_2: {only_in_hosts}\n"
        return result

In [6]:
df_athletes = load_table_from_mysql('olympic_athletes')
df_hosts = load_table_from_mysql('olympic_hosts')
df_medals = load_table_from_mysql('olympic_medals')
df_results = load_table_from_mysql('olympic_results')

  df_pandas = pd.read_sql(query, connection)



In [ ]:
df_hosts

,index,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
2,2,pyeongchang-2018,2018-02-25T08:00:00Z,2018-02-08T23:00:00Z,Republic of Korea,PyeongChang 2018,Winter,2018
3,3,rio-2016,2016-08-21T21:00:00Z,2016-08-05T12:00:00Z,Brazil,Rio 2016,Summer,2016
4,4,sochi-2014,2014-02-23T16:00:00Z,2014-02-07T04:00:00Z,Russian Federation,Sochi 2014,Winter,2014
5,5,london-2012,2012-08-12T19:00:00Z,2012-07-27T07:00:00Z,Great Britain,London 2012,Summer,2012
6,6,vancouver-2010,2010-02-28T04:00:00Z,2010-02-12T16:00:00Z,Canada,Vancouver 2010,Winter,2010
7,7,beijing-2008,2008-08-24T12:00:00Z,2008-08-08T00:00:00Z,China,Beijing 2008,Summer,2008
8,8,turin-2006,2006-02-26T19:00:00Z,2006-02-10T07:00:00Z,Italy,Turin 2006,Winter,2006
9,9,athens-2004,2004-08-29T18:00:00Z,2004-08-13T06:00:00Z,Greece,Athens 2004,Summer,2004


In [ ]:
df_results

,index,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_3_letter_code,athlete_full_name
0,0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",0.0,1,Italy,ITA,None
1,1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",0.0,2,Norway,NOR,None
2,2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",0.0,3,Sweden,SWE,None
3,3,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jennifer DODDS', 'https://olympics.com/en/a...",0.0,4,Great Britain,GBR,None
4,4,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",0.0,5,Canada,CAN,None
...,...,...,...,...,...,...,...,...,...,...,...,...
158921,158921,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,Poland,POL,Franciszek BUJAK
158922,158922,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,Poland,POL,Henryk Mückenbrunn
158923,158923,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,Czechoslovakia,TCH,Milda Prokopec
158924,158924,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,United States of America,USA,Sigurd Overby


In [ ]:
df_results.isnull().sum()

index                         0
discipline_title              0
event_title                   0
slug_game                     0
participant_type              0
medal_type               138720
athletes                 150951
rank_equal               126527
rank_position                 0
country_name                  0
country_3_letter_code         0
athlete_full_name         21108
dtype: int64

In [ ]:
df_hosts.isnull().sum()

index              0
game_slug          0
game_end_date      0
game_start_date    0
game_location      0
game_name          0
game_season        0
game_year          0
dtype: int64

In [ ]:
df_results.slug_game.unique()

array(['beijing-2022', 'tokyo-2020', 'pyeongchang-2018', 'rio-2016',
       'sochi-2014', 'london-2012', 'vancouver-2010', 'beijing-2008',
       'turin-2006', 'athens-2004', 'salt-lake-city-2002', 'sydney-2000',
       'nagano-1998', 'atlanta-1996', 'lillehammer-1994',
       'barcelona-1992', 'albertville-1992', 'seoul-1988', 'calgary-1988',
       'los-angeles-1984', 'sarajevo-1984', 'moscow-1980',
       'lake-placid-1980', 'montreal-1976', 'innsbruck-1976',
       'munich-1972', 'sapporo-1972', 'mexico-city-1968', 'grenoble-1968',
       'tokyo-1964', 'innsbruck-1964', 'rome-1960', 'squaw-valley-1960',
       'melbourne-1956', 'cortina-d-ampezzo-1956', 'helsinki-1952',
       'oslo-1952', 'london-1948', 'st-moritz-1948', 'berlin-1936',
       'garmisch-partenkirchen-1936', 'los-angeles-1932',
       'lake-placid-1932', 'amsterdam-1928', 'st-moritz-1928',
       'paris-1924', 'chamonix-1924', 'antwerp-1920', 'stockholm-1912',
       'london-1908', 'st-louis-1904', 'paris-1900', 'at

In [ ]:
df_hosts.game_slug.unique()

array(['beijing-2022', 'tokyo-2020', 'pyeongchang-2018', 'rio-2016',
       'sochi-2014', 'london-2012', 'vancouver-2010', 'beijing-2008',
       'turin-2006', 'athens-2004', 'salt-lake-city-2002', 'sydney-2000',
       'nagano-1998', 'atlanta-1996', 'lillehammer-1994',
       'barcelona-1992', 'albertville-1992', 'seoul-1988', 'calgary-1988',
       'los-angeles-1984', 'sarajevo-1984', 'moscow-1980',
       'lake-placid-1980', 'montreal-1976', 'innsbruck-1976',
       'munich-1972', 'sapporo-1972', 'mexico-city-1968', 'grenoble-1968',
       'tokyo-1964', 'innsbruck-1964', 'rome-1960', 'squaw-valley-1960',
       'melbourne-1956', 'cortina-d-ampezzo-1956', 'helsinki-1952',
       'oslo-1952', 'london-1948', 'st-moritz-1948', 'berlin-1936',
       'garmisch-partenkirchen-1936', 'los-angeles-1932',
       'lake-placid-1932', 'amsterdam-1928', 'st-moritz-1928',
       'paris-1924', 'chamonix-1924', 'antwerp-1920', 'stockholm-1912',
       'london-1908', 'st-louis-1904', 'paris-1900', 'at

In [ ]:
unique_slug_game = df_results['slug_game'].unique()
unique_game_slug = df_hosts['game_slug'].unique()

set_slug_game = set(unique_slug_game)
set_game_slug = set(unique_game_slug)

if set_slug_game.issubset(set_game_slug) and set_game_slug.issubset(set_slug_game):
    print("Toutes les valeurs correspondent")
else:
    print("Certaines valeurs ne correspondent pas")
    only_in_results = set_slug_game.difference(set_game_slug)
    if only_in_results:
        print(f"Valeurs uniquement dans df_results: {only_in_results}")
    only_in_hosts = set_game_slug.difference(set_slug_game)
    if only_in_hosts:
        print(f"Valeurs uniquement dans df_hosts: {only_in_hosts}")

Toutes les valeurs correspondent


In [9]:
df_results_hosts = pd.merge(df_results, df_hosts, left_on='slug_game', right_on='game_slug', how='left')
df_results_hosts

,index_x,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_3_letter_code,athlete_full_name,index_y,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
0,0,Curling,Mixed Doubles,beijing-2022,GameTeam,GOLD,"[('Stefania CONSTANTINI', 'https://olympics.co...",0.0,1,Italy,ITA,None,0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
1,1,Curling,Mixed Doubles,beijing-2022,GameTeam,SILVER,"[('Kristin SKASLIEN', 'https://olympics.com/en...",0.0,2,Norway,NOR,None,0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
2,2,Curling,Mixed Doubles,beijing-2022,GameTeam,BRONZE,"[('Almida DE VAL', 'https://olympics.com/en/at...",0.0,3,Sweden,SWE,None,0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
3,3,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Jennifer DODDS', 'https://olympics.com/en/a...",0.0,4,Great Britain,GBR,None,0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
4,4,Curling,Mixed Doubles,beijing-2022,GameTeam,None,"[('Rachel HOMAN', 'https://olympics.com/en/ath...",0.0,5,Canada,CAN,None,0,beijing-2022,2022-02-20T12:00:00Z,2022-02-04T15:00:00Z,China,Beijing 2022,Winter,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158921,158921,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,Poland,POL,Franciszek BUJAK,46,chamonix-1924,1924-02-05T20:00:00Z,1924-01-25T08:00:00Z,France,Chamonix 1924,Winter,1924
158922,158922,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,Poland,POL,Henryk Mückenbrunn,46,chamonix-1924,1924-02-05T20:00:00Z,1924-01-25T08:00:00Z,France,Chamonix 1924,Winter,1924
158923,158923,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,Czechoslovakia,TCH,Milda Prokopec,46,chamonix-1924,1924-02-05T20:00:00Z,1924-01-25T08:00:00Z,France,Chamonix 1924,Winter,1924
158924,158924,Ski Jumping,Normal Hill Individual men,chamonix-1924,Athlete,None,None,NaN,DNS,United States of America,USA,Sigurd Overby,46,chamonix-1924,1924-02-05T20:00:00Z,1924-01-25T08:00:00Z,France,Chamonix 1924,Winter,1924


In [10]:
df_results_hosts_summer = df_results_hosts[df_results_hosts.game_season == 'Summer']
df_results_hosts_summer

,index_x,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_3_letter_code,athlete_full_name,index_y,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
3774,3774,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,GOLD,"[('Fatima GALVEZ', 'https://olympics.com/en/at...",0.0,1,Spain,ESP,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3775,3775,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,SILVER,"[('Alessandra PERILLI', 'https://olympics.com/...",0.0,2,San Marino,SMR,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3776,3776,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,BRONZE,"[('Madelynn Ann BERNAU', 'https://olympics.com...",0.0,3,United States of America,USA,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3777,3777,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,None,"[('Zuzana STEFECEKOVA', 'https://olympics.com/...",0.0,4,Slovakia,SVK,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3778,3778,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,None,"[('Yukie NAKAYAMA', 'https://olympics.com/en/a...",0.0,5,Japan,JPN,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158863,158863,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,40,Mexico,MEX,Luis RAZO,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976
158864,158864,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,41,France,FRA,Marc ROGUET,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976
158865,158865,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,42,Japan,JPN,Ryuichi OBATA,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976
158866,158866,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,43,Belgium,BEL,Eric WAUTERS,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976


In [11]:
df_results_hosts_summer_clean = df_results_hosts_summer[["medal_type", "country_name", "game_slug", "game_location", "game_year"]]
df_results_hosts_summer_clean

,medal_type,country_name,game_slug,game_location,game_year
3774,GOLD,Spain,tokyo-2020,Japan,2020
3775,SILVER,San Marino,tokyo-2020,Japan,2020
3776,BRONZE,United States of America,tokyo-2020,Japan,2020
3777,None,Slovakia,tokyo-2020,Japan,2020
3778,None,Japan,tokyo-2020,Japan,2020
...,...,...,...,...,...
158863,None,Mexico,montreal-1976,Canada,1976
158864,None,France,montreal-1976,Canada,1976
158865,None,Japan,montreal-1976,Canada,1976
158866,None,Belgium,montreal-1976,Canada,1976


In [ ]:
print(comparer_column(df_results_hosts_summer_clean['country_name'], df_results_hosts_summer_clean['game_location']))

Certaines valeurs ne correspondent pas
Valeurs uniquement dans df_1: {'Dominican Republic', 'Bolivia', 'Gambia', 'Haiti', 'Islamic Republic of Iran', 'Nepal', 'Liberia', 'Central African Republic', 'Burkina Faso', 'Yugoslavia', 'Yemen Democratic Republic', 'Mali', 'Georgia', 'Cuba', 'South Vietnam', 'South Sudan', 'Bulgaria', 'Maldives', 'Saint Lucia', 'Nauru', 'Bangladesh', 'Nigeria', 'Somalia', 'Bermuda', 'Jamaica', 'Guinea-Bissau', 'Guatemala', 'Iraq', 'Albania', 'Denmark', 'Ecuador', 'Slovenia', 'San Marino', 'Zambia', 'Virgin Islands, US', 'Singapore', 'Jordan', 'Iceland', 'Vanuatu', 'Virgin Islands, British', 'Czech Republic', 'Chinese Taipei', 'Antigua and Barbuda', 'Cape Verde', 'New Zealand', 'Ghana', 'Costa Rica', 'Puerto Rico', 'North Borneo', 'Ireland', 'Latvia', 'Hungary', 'Refugee Olympic Athletes', 'Fiji', 'Uzbekistan', 'Sri Lanka', 'Newfoundland', 'Argentina', 'Zimbabwe', 'Aruba', 'Cayman Islands', 'Bahrain', 'Ivory Coast', 'Romania', 'Kazakhstan', 'Portugal', 'Qatar', 

In [12]:
remplacement_dict_game_location = {
  "USSR" : "Russian Federation",
  "United States" : "United States of America",
  "Australia, Sweden" : "Australia",
  "China" : "People's Republic of China",
}

df_results_hosts_summer_clean["game_location"] = df_results_hosts_summer_clean["game_location"].replace(remplacement_dict_game_location)
print(comparer_column(df_results_hosts_summer_clean['country_name'], df_results_hosts_summer_clean['game_location']))
print(df_results_hosts_summer_clean['country_name'].unique())
print(df_results_hosts_summer_clean['game_location'].unique())



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




Certaines valeurs ne correspondent pas
Valeurs uniquement dans df_1: {'Saint Kitts and Nevis', 'Thailand', 'Hong Kong, China', 'Refugee Olympic Athletes', 'Estonia', 'Djibouti', 'Ukraine', 'Cape Verde', 'Monaco', 'Kyrgyzstan', 'Cameroon', 'Bosnia and Herzegovina', 'Philippines', 'Cambodia', 'Burkina Faso', 'Bahamas', 'Islamic Republic of Iran', 'Lithuania', 'Costa Rica', 'Republic of Moldova', 'North Macedonia', 'Palestine', 'Independent Olympic Athletes', 'Cayman Islands', 'Azerbaijan', 'Syrian Arab Republic', 'Slovakia', "Côte d'Ivoire", 'Togo', 'El Salvador', 'Belize', 'Eritrea', 'Barbados', 'Rwanda', 'Federated States of Micronesia', 'Bhutan', 'Turkmenistan', 'Netherlands Antilles', 'Iceland', 'Yemen Democratic Republic', 'Mozambique', 'Oman', 'Tonga', 'Malawi', 'Luxembourg', 'Slovenia', 'Nauru', 'Pakistan', 'United Arab Emirates', 'Uruguay', 'Liberia', 'Central African Republic', 'Guatemala', 'Argentina', 'Yugoslavia', 'Comoros', 'Portugal', 'New Zealand', 'ROC', 'Switzerland', 'E

In [ ]:
# Ajout du  participates_on_home_soil

In [13]:
df_results_hosts_summer_clean['participates_on_home_soil'] = df_results_hosts_summer_clean['country_name'] == df_results_hosts_summer_clean['game_location']
df_results_hosts_summer_clean



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




,medal_type,country_name,game_slug,game_location,game_year,participates_on_home_soil
3774,GOLD,Spain,tokyo-2020,Japan,2020,False
3775,SILVER,San Marino,tokyo-2020,Japan,2020,False
3776,BRONZE,United States of America,tokyo-2020,Japan,2020,False
3777,None,Slovakia,tokyo-2020,Japan,2020,False
3778,None,Japan,tokyo-2020,Japan,2020,True
...,...,...,...,...,...,...
158863,None,Mexico,montreal-1976,Canada,1976,False
158864,None,France,montreal-1976,Canada,1976,False
158865,None,Japan,montreal-1976,Canada,1976,False
158866,None,Belgium,montreal-1976,Canada,1976,False


In [14]:
df_results_hosts_summer_clean[(df_results_hosts_summer_clean['country_name'] == "United States of America") &
                        (df_results_hosts_summer_clean['participates_on_home_soil'] == True)]

,medal_type,country_name,game_slug,game_location,game_year,participates_on_home_soil
66483,BRONZE,United States of America,atlanta-1996,United States of America,1996,True
66535,None,United States of America,atlanta-1996,United States of America,1996,True
66544,None,United States of America,atlanta-1996,United States of America,1996,True
66584,None,United States of America,atlanta-1996,United States of America,1996,True
66619,None,United States of America,atlanta-1996,United States of America,1996,True
...,...,...,...,...,...,...
156944,None,United States of America,st-louis-1904,United States of America,1904,True
156946,SILVER,United States of America,st-louis-1904,United States of America,1904,True
156948,GOLD,United States of America,st-louis-1904,United States of America,1904,True
156949,SILVER,United States of America,st-louis-1904,United States of America,1904,True


In [15]:
df_results_hosts_summer_clean.isnull().sum()

medal_type                   97814
country_name                     0
game_slug                        0
game_location                    0
game_year                        0
participates_on_home_soil        0
dtype: int64

In [16]:
df_results_hosts_summer_clean.medal_type.unique()

array(['GOLD', 'SILVER', 'BRONZE', None], dtype=object)

In [17]:
medal_counts = df_results_hosts_summer_clean.groupby(['game_year', 'country_name', 'medal_type']).size().unstack(fill_value=0)
medal_counts = medal_counts.rename(columns={'GOLD': 'gold_medals', 'SILVER': 'silver_medals', 'BRONZE': 'bronze_medals'})

medal_counts = medal_counts[['gold_medals', 'silver_medals', 'bronze_medals']]
medal_counts = medal_counts.reset_index()

home_soil = df_results_hosts_summer_clean[['game_year', 'country_name', 'participates_on_home_soil']].drop_duplicates()

medal_counts = pd.merge(medal_counts, home_soil, on=['game_year', 'country_name'], how='left')
medal_counts

,game_year,country_name,gold_medals,silver_medals,bronze_medals,participates_on_home_soil
0,1896,Australia,2,0,0,False
1,1896,Austria,2,1,2,False
2,1896,Denmark,1,2,3,False
3,1896,France,5,4,2,False
4,1896,Germany,6,5,2,False
...,...,...,...,...,...,...
1336,2020,Côte d'Ivoire,0,0,1,False
1337,2020,North Macedonia,0,1,0,False
1338,2020,ROC,20,28,22,False
1339,2020,San Marino,0,1,2,False


In [18]:
medal_counts["total_medals"] = medal_counts["gold_medals"] + medal_counts["silver_medals"] + medal_counts["bronze_medals"]
medal_counts

,game_year,country_name,gold_medals,silver_medals,bronze_medals,participates_on_home_soil,total_medals
0,1896,Australia,2,0,0,False,2
1,1896,Austria,2,1,2,False,5
2,1896,Denmark,1,2,3,False,6
3,1896,France,5,4,2,False,11
4,1896,Germany,6,5,2,False,13
...,...,...,...,...,...,...,...
1336,2020,Côte d'Ivoire,0,0,1,False,1
1337,2020,North Macedonia,0,1,0,False,1
1338,2020,ROC,20,28,22,False,70
1339,2020,San Marino,0,1,2,False,3


In [21]:
df_results_hosts_summer

,index_x,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_3_letter_code,athlete_full_name,index_y,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year
3774,3774,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,GOLD,"[('Fatima GALVEZ', 'https://olympics.com/en/at...",0.0,1,Spain,ESP,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3775,3775,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,SILVER,"[('Alessandra PERILLI', 'https://olympics.com/...",0.0,2,San Marino,SMR,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3776,3776,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,BRONZE,"[('Madelynn Ann BERNAU', 'https://olympics.com...",0.0,3,United States of America,USA,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3777,3777,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,None,"[('Zuzana STEFECEKOVA', 'https://olympics.com/...",0.0,4,Slovakia,SVK,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
3778,3778,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,None,"[('Yukie NAKAYAMA', 'https://olympics.com/en/a...",0.0,5,Japan,JPN,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158863,158863,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,40,Mexico,MEX,Luis RAZO,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976
158864,158864,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,41,France,FRA,Marc ROGUET,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976
158865,158865,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,42,Japan,JPN,Ryuichi OBATA,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976
158866,158866,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,43,Belgium,BEL,Eric WAUTERS,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976


In [22]:
df_results_hosts_summer.participant_type.unique()

array(['GameTeam', 'Athlete'], dtype=object)

In [27]:
df_corr = medal_counts.copy()
mapping = {'GameTeam': 0, 'Athlete': 1}
participant_type_code = df_results_hosts_summer['participant_type'].map(mapping).astype(int)
print(participant_type_code)
# df_corr["participant_type_code"] = participant_type_code
# df_corr['participates_on_home_soil'] = df_corr['participates_on_home_soil'].astype(int)
# df_corr

3774      0
3775      0
3776      0
3777      0
3778      0
         ..
158863    1
158864    1
158865    1
158866    1
158925    0
Name: participant_type, Length: 114524, dtype: int64


In [35]:
df_corr = df_results_hosts_summer.copy()
mapping = {'GameTeam': 0, 'Athlete': 1}
participant_type_code = df_results_hosts_summer['participant_type'].map(mapping).astype(int)
print("Indices de df_results_hosts_summer :", df_results_hosts_summer.index)
print("Indices de participant_type_code :", participant_type_code.index)
participant_type_code = participant_type_code.reindex(df_corr.index)
df_corr['participant_type_code'] = participant_type_code
df_corr
# Vous pouvez ensuite convertir d'autres colonnes en types numériques si nécessaire
# df_corr['some_other_column'] = df_corr['some_other_column'].astype(int)


Indices de df_results_hosts_summer : Index([  3774,   3775,   3776,   3777,   3778,   3779,   3780,   3781,   3782,
         3783,
       ...
       158858, 158859, 158860, 158861, 158862, 158863, 158864, 158865, 158866,
       158925],
      dtype='int64', length=114524)
Indices de participant_type_code : Index([  3774,   3775,   3776,   3777,   3778,   3779,   3780,   3781,   3782,
         3783,
       ...
       158858, 158859, 158860, 158861, 158862, 158863, 158864, 158865, 158866,
       158925],
      dtype='int64', length=114524)


,index_x,discipline_title,event_title,slug_game,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,...,athlete_full_name,index_y,game_slug,game_end_date,game_start_date,game_location,game_name,game_season,game_year,participant_type_code
3774,3774,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,GOLD,"[('Fatima GALVEZ', 'https://olympics.com/en/at...",0.0,1,Spain,...,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020,0
3775,3775,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,SILVER,"[('Alessandra PERILLI', 'https://olympics.com/...",0.0,2,San Marino,...,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020,0
3776,3776,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,BRONZE,"[('Madelynn Ann BERNAU', 'https://olympics.com...",0.0,3,United States of America,...,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020,0
3777,3777,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,None,"[('Zuzana STEFECEKOVA', 'https://olympics.com/...",0.0,4,Slovakia,...,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020,0
3778,3778,Shooting,Trap Mixed Team,tokyo-2020,GameTeam,None,"[('Yukie NAKAYAMA', 'https://olympics.com/en/a...",0.0,5,Japan,...,None,1,tokyo-2020,2021-08-08T14:00:00Z,2021-07-23T11:00:00Z,Japan,Tokyo 2020,Summer,2020,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158863,158863,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,40,Mexico,...,Luis RAZO,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976,1
158864,158864,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,41,France,...,Marc ROGUET,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976,1
158865,158865,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,42,Japan,...,Ryuichi OBATA,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976,1
158866,158866,Equestrian Jumping,Jumping Individual mixed,montreal-1976,Athlete,None,None,NaN,43,Belgium,...,Eric WAUTERS,23,montreal-1976,1976-07-31T22:00:00Z,1976-07-17T12:00:00Z,Canada,Montreal 1976,Summer,1976,1


In [20]:
df_numeric = df_corr.select_dtypes(include=[float, int])
correlation_matrix = df_numeric.corr()

fig = px.imshow(correlation_matrix,
                labels=dict(x="Variables", y="Variables", color="Corrélation"),
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1)

fig.update_layout(title="Matrice de Corrélation", width=800, height=800)
fig.show()

In [ ]:
medal_counts[(medal_counts['country_name'] == "United States of America") &
                        (medal_counts['participates_on_home_soil'] == True)]

,game_year,country_name,gold_medals,silver_medals,bronze_medals,participates_on_home_soil,total_medals
41,1904,United States of America,76,78,77,True,231
172,1932,United States of America,41,32,30,True,103
601,1984,United States of America,83,61,30,True,174
766,1996,United States of America,44,32,25,True,101


In [ ]:
# United States of America
US_medals = medal_counts[medal_counts["country_name"] == "United States of America"]

fig = px.line(US_medals, x='game_year', y='total_medals', color='country_name',
              title='Total Medals by Year for United States of America',
              labels={'game_year': 'Year', 'total_medals': 'Total Medals'},
              markers=True)

for year in US_medals[US_medals['participates_on_home_soil']]['game_year']:
    fig.add_vline(x=year, line=dict(color='Red', width=2, dash='dash'), name=f'Home Soil {year}')

fig.update_layout(legend=dict(title='Country/Participation'),
                  legend_traceorder="normal")

fig.show()

In [ ]:
france_medals = medal_counts[medal_counts["country_name"] == "France"]
fig = px.line(france_medals, x='game_year', y='total_medals', color='country_name',
              title='Total Medals by Year for France',
              labels={'game_year': 'Year', 'total_medals': 'Total Medals'},
              markers=True)

for year in france_medals[france_medals['participates_on_home_soil']]['game_year']:
    fig.add_vline(x=year, line=dict(color='Red', width=2, dash='dash'), name=f'Home Soil {year}')

fig.update_layout(legend=dict(title='Country/Participation'),
                  legend_traceorder="normal")

fig.show()

In [ ]:
fig = px.line(medal_counts, x='game_year', y='total_medals', color='country_name',
              title='Total Medals by Country and Year',
              labels={'game_year': 'Year', 'total_medals': 'Total Medals'},
              markers=True)

fig.show()

In [28]:
countries_with_home_participation = medal_counts[medal_counts['participates_on_home_soil'] == True]['country_name'].unique()
filtered_medal_counts = medal_counts[medal_counts['country_name'].isin(countries_with_home_participation)]
filtered_medal_counts

,game_year,country_name,gold_medals,silver_medals,bronze_medals,participates_on_home_soil,total_medals
0,1896,Australia,2,0,0,False,2
3,1896,France,5,4,2,False,11
4,1896,Germany,6,5,2,False,13
5,1896,Great Britain,2,3,2,False,7
6,1896,Greece,10,18,19,True,47
...,...,...,...,...,...,...,...
1269,2020,Brazil,6,6,8,False,20
1271,2020,Japan,27,13,17,True,57
1281,2020,Mexico,0,0,4,False,4
1286,2020,Republic of Korea,6,4,10,False,20


In [ ]:
# filtered_medal_counts[filtered_medal_counts["game_year"] > 1910]
performance = filtered_medal_counts.groupby(['country_name', 'participates_on_home_soil'])['total_medals'].mean().unstack(fill_value=0)
performance = performance.rename(columns={True: 'home_soil', False: 'away'})
performance['difference'] = performance['home_soil'] - performance['away']

fig = px.bar(performance.reset_index(), x='country_name', y='difference',
             title='Difference in Performance on Home Soil vs Away for Each Country',
             labels={'country_name': 'Country', 'difference': 'Difference in Total Medals'},
             color='difference', color_continuous_scale='Viridis')

fig.show()

In [ ]:
# Nombre d'athlètes participant aux JO 2024
# Médaille totale et distribution par type

In [ ]:
medal_counts

,game_year,country_name,gold_medals,silver_medals,bronze_medals,participates_on_home_soil,total_medals
0,1896,Australia,2,0,0,False,2
1,1896,Austria,2,1,2,False,5
2,1896,Denmark,1,2,3,False,6
3,1896,France,5,4,2,False,11
4,1896,Germany,6,5,2,False,13
...,...,...,...,...,...,...,...
1336,2020,Côte d'Ivoire,0,0,1,False,1
1337,2020,North Macedonia,0,1,0,False,1
1338,2020,ROC,20,28,22,False,70
1339,2020,San Marino,0,1,2,False,3


In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objs as go


df = medal_counts.copy()

liste_pays = list(df.country_name.unique())
dict_pays = {pays: idx for idx, pays in enumerate(liste_pays)}
df['country_id'] = df['country_name'].map(dict_pays)

df['participates_on_home_soil'] = df['participates_on_home_soil'].astype(int)

X = df[['game_year', 'country_id', 'participates_on_home_soil']]
y = df['total_medals']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
df

,game_year,country_name,gold_medals,silver_medals,bronze_medals,participates_on_home_soil,total_medals,country_id
0,1896,Australia,2,0,0,0,2,0
1,1896,Austria,2,1,2,0,5,1
2,1896,Denmark,1,2,3,0,6,2
3,1896,France,5,4,2,0,11,3
4,1896,Germany,6,5,2,0,13,4
...,...,...,...,...,...,...,...,...
1336,2020,Côte d'Ivoire,0,0,1,0,1,149
1337,2020,North Macedonia,0,1,0,0,1,150
1338,2020,ROC,20,28,22,0,70,151
1339,2020,San Marino,0,1,2,0,3,152


In [31]:
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mae')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=0)
y_pred_tf = model.predict(X_test)

mae_tf = mean_absolute_error(y_test, y_pred_tf)


9/9 [==============================] - 0s 2ms/step


In [32]:
pred_2024 = pd.DataFrame({
    'game_year': [2024],
    'country_id': [dict_pays['France']],
    'participates_on_home_soil': [1]
})

pred_2024 = pred_2024.values

pred_2024_tf = model.predict(pred_2024)

print(f"Prediction: {pred_2024_tf}")


1/1 [==============================] - 0s 24ms/step
Prediction: [[18.974379]]


In [45]:
model.save('my_model.keras')